In [92]:
import sys
import subprocess

# implement pip as a subprocess:
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 
'matplotlib'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 
'numpy'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 
'scipy'])

import random
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from matplotlib.widgets import Slider, Button


In [93]:
x = np.random.normal(5.5, 3, 25)

In [94]:
print(x)

[10.86081113 12.95742969  7.9181162   4.20226865  6.12813902  3.15680428
  1.8169566   4.86362945  9.14479028  4.85433536  4.6799922   1.95146025
  2.08643061  8.44351461 10.14860403 10.08276849  4.63085054  9.51849997
 10.59119192  5.38378193  3.07036133  6.14762981  8.9132151   4.9555681
  7.98212909]


In [95]:
alpha = np.random.gamma(1)
beta = np.random.uniform(0, 2)
Y = alpha*x + beta + np.random.normal(0, 1, 25)

In [96]:
print(Y)

[10.06460702 12.79608088  8.97268402  5.79048339  6.71750059  4.59118264
  3.6163066   7.48053248 10.15375544  5.90514415  4.57649326  1.55029606
  1.89899656  9.95937886 10.95138511 10.57802601  4.12486479 11.79806785
 10.22023269  7.77446516  3.86289502  6.21846565  9.83395165  5.2487127
  7.38456688]


In [97]:

plt.plot(x, Y, lw=0, marker='o')

In [98]:

res = st.linregress(x, Y)

plt.plot(x, Y, lw=0, marker='o', label='Data')
plt.plot(x, res.slope*x + res.intercept, color='red', label='Linear regression fitted model')
plt.plot(x, alpha*x + beta, color='blue', lw = 1, ls='--', label='Ground truth')
plt.vlines(x, Y, res.slope*x + res.intercept, alpha=0.3)

plt.legend()

In [66]:
def loss_function(x, Y, slope, intercept):
    return np.sum((Y - slope*x - intercept)**2)
    

In [99]:
from matplotlib.widgets import Slider, Button

# Define initial parameters
init_slope = 0
init_intercept = np.mean(Y)
best_slope, best_intercept = init_slope, init_intercept

# Create the figure and the line that we will manipulate
fig, ax = plt.subplots()
scatter_points = plt.plot(x, Y, lw=0, marker='o')
line, = plt.plot(x, init_slope*x + init_intercept, color='red', label='Current fit')
best_line, = plt.plot(x, init_slope*x + init_intercept, color='blue', label="Best fit")
vlines = plt.vlines(x, Y, init_slope*x + init_intercept, alpha=0.3)


ax.set_xlabel('x')
ax.legend()

# adjust the main plot to make room for the sliders
plt.subplots_adjust(left=0.25, right=0.75, bottom=0.25)

# Make a horizontal slider to control the slope.
ax_slope = plt.axes([0.25, 0.1, 0.5, 0.03])
slope_slider = Slider(
    ax=ax_slope,
    label='Slope',
    valmin=-3,
    valmax=3,
    valinit=init_slope,
)

# Make a vertically oriented slider to control the intercept
ax_intercept = plt.axes([0.1, 0.25, 0.0225, 0.63])
intercept_slider = Slider(
    ax=ax_intercept,
    label="Intercept",
    valmin=-5,
    valmax=25,
    valinit=init_intercept,
    orientation="vertical"
)

ax_text = plt.axes([0.76, 0.1, 0.22, 0.8])
least_error_text = "Least error achieved: {:.2f}".format(loss_function(x, Y, best_slope, best_intercept))
best_para_text = "Best (slope, intercept) = ({0:.2f}, {1:.2f})".format(best_slope, best_intercept)
ax_text.set_axis_off()
curr_error = ax_text.text(0.0, 0.9, "Current error: {:.2f}".format(loss_function(x, Y, init_slope, init_intercept)))
le_display = ax_text.text(0.0, 0.7, least_error_text, wrap=True)
bp_display = ax_text.text(0.0, 0.5, best_para_text, wrap=True)

# The function to be called anytime a slider's value changes
def update(val):
    global best_intercept, best_slope
    line.set_ydata(slope_slider.val*x + intercept_slider.val)
    vlines.set_segments(np.array([[[x[i], Y[i]], [x[i], slope_slider.val*x[i] + intercept_slider.val]] for i in range(len(x))]))
    
    if loss_function(x, Y, slope_slider.val, intercept_slider.val) < loss_function(x, Y, best_slope, best_intercept):
        best_slope, best_intercept = slope_slider.val, intercept_slider.val
        best_line.set_ydata(best_slope*x + best_intercept)
    least_error_text = "Least error achieved: {:.2f}".format(loss_function(x, Y, best_slope, best_intercept))
    curr_error.set_text("Current error: {:.2f}".format(loss_function(x, Y, slope_slider.val, intercept_slider.val)))
    best_para_text = "Best (slope, intercept) = ({0:.2f}, {1:.2f})".format(best_slope, best_intercept)
    le_display.set_text(least_error_text)
    bp_display.set_text(best_para_text)
    fig.canvas.draw_idle()


# register the update function with each slider
slope_slider.on_changed(update)
intercept_slider.on_changed(update)

# Create a `matplotlib.widgets.Button` to reset the sliders to initial values.
resetax = plt.axes([0.8, 0.025, 0.1, 0.04])
button = Button(resetax, 'Reset', hovercolor='0.975')


def reset(event):
    slope_slider.reset()
    intercept_slider.reset()
button.on_clicked(reset)

plt.show()

%matplotlib qt

In [22]:
segs = vlines.get_segments()
print([seg for seg in segs])

[array([[ 6.13053274, 11.10849035],
       [ 6.13053274, 11.61401858]]), array([[ 5.52780471,  9.93637627],
       [ 5.52780471, 11.61401858]]), array([[ 4.74004249, 10.24429617],
       [ 4.74004249, 11.61401858]]), array([[10.7303704 , 22.26224846],
       [10.7303704 , 11.61401858]]), array([[ 9.3805904 , 19.75754484],
       [ 9.3805904 , 11.61401858]]), array([[ 6.72160749, 13.51665347],
       [ 6.72160749, 11.61401858]]), array([[ 4.36010031,  7.92831311],
       [ 4.36010031, 11.61401858]]), array([[ 5.83080735, 11.61428684],
       [ 5.83080735, 11.61401858]]), array([[ 6.42411728, 11.90255293],
       [ 6.42411728, 11.61401858]]), array([[ 9.30494547, 18.95932476],
       [ 9.30494547, 11.61401858]]), array([[ 7.31878908, 14.29289538],
       [ 7.31878908, 11.61401858]]), array([[ 7.14011101, 19.09668321],
       [ 7.14011101, 11.61401858]]), array([[ 3.94918451,  5.72040553],
       [ 3.94918451, 11.61401858]]), array([[ 4.88908354, 11.90031805],
       [ 4.88908354, 11.6140